# Example 1: Simplest Case

In [1]:
from examples.example_6_simplest_case_storewrapper.ex_6_model_classes_with_vidigi_logging import Trial, g
from vidigi.animation import animate_activity_log
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"
import os

In [2]:
#| echo: false
#| output: asis
# Path to the external Python script
file_path = "ex_6_model_classes_with_vidigi_logging.py"

# Read the file content
if os.path.exists(file_path):
    with open(file_path, "r") as f:
        code_content = f.read()
else:
    code_content = "File not found."
with open(file_path, "r") as f:
    code_content = f.read()

# Print the Quarto `{details}` block for collapsible output
print(f"""
:::{{.callout-note collapse="true"}}
### View Imported Code, which has had logging steps added at the appropriate points in the 'model' class

```python
{code_content}
```

:::

""")


:::{.callout-note collapse="true"}
### View Imported Code, which has had logging steps added at the appropriate points in the 'model' class

```python
import random
import numpy as np
import pandas as pd
import simpy
from sim_tools.distributions import Exponential, Lognormal
from vidigi.utils import populate_store, StoreResourceLikeWrapper


# Class to store global parameter values.  We don't create an instance of this
# class - we just refer to the class blueprint itself to access the numbers
# inside.
class g:
    '''
    Create a scenario to parameterise the simulation model

    Parameters:
    -----------
    random_number_set: int, optional (default=DEFAULT_RNG_SET)
        Set to control the initial seeds of each stream of pseudo
        random numbers used in the model.

    n_cubicles: int
        The number of treatment cubicles

    trauma_treat_mean: float
        Mean of the trauma cubicle treatment distribution (Lognormal)

    trauma_treat_var: float
        Variance of

In [3]:
my_trial = Trial()

my_trial.run_trial()

4 nurses



TypeError: 'StoreRequestWrapper' object is not iterable

In [ ]:
my_trial.all_event_logs.head(50)

In [ ]:
event_position_df = pd.DataFrame([
                    {'event': 'arrival',
                     'x':  50, 'y': 300,
                     'label': "Arrival" },

                    # Triage - minor and trauma
                    {'event': 'treatment_wait_begins',
                     'x':  205, 'y': 275,
                     'label': "Waiting for Treatment"},

                    {'event': 'treatment_begins',
                     'x':  205, 'y': 175,
                     'resource':'n_cubicles',
                     'label': "Being Treated"},

                    {'event': 'exit',
                     'x':  270, 'y': 70,
                     'label': "Exit"}

                ])


In [ ]:
animate_activity_log(
        event_log=my_trial.all_event_logs[my_trial.all_event_logs['run']==1],
        event_position_df= event_position_df,
        scenario=g(),
        debug_mode=True,
        setup_mode=False,
        every_x_time_units=1,
        include_play_button=True,
        icon_and_text_size=20,
        gap_between_entities=6,
        gap_between_rows=25,
        plotly_height=700,
        frame_duration=200,
        plotly_width=1200,
        override_x_max=300,
        override_y_max=500,
        limit_duration=g.sim_duration,
        wrap_queues_at=25,
        step_snapshot_max=125,
        time_display_units="dhm",
        display_stage_labels=False,
        add_background_image="https://raw.githubusercontent.com/Bergam0t/vidigi/refs/heads/main/examples/example_1_simplest_case/Simplest%20Model%20Background%20Image%20-%20Horizontal%20Layout.drawio.png",
    )